In [9]:
from flask import Flask, redirect, url_for, render_template, request
import lime
from lime import lime_image

In [10]:
app = Flask(__name__)

In [11]:
@app.route("/")
def home():
    return render_template("index.html")

In [12]:
@app.route("/explain")
def explain():
    return render_template("explanation.html")

In [ ]:
if __name__ == "__main__":
    app.run(host="0.0.0.0", port=3000)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://0.0.0.0:3000/ (Press CTRL+C to quit)
[2021-08-01 06:32:46,222] ERROR in app: Exception on / [GET]
Traceback (most recent call last):
  File "/home/jackie/anaconda3/envs/lime-env/lib/python3.7/site-packages/flask/app.py", line 2447, in wsgi_app
    response = self.full_dispatch_request()
  File "/home/jackie/anaconda3/envs/lime-env/lib/python3.7/site-packages/flask/app.py", line 1952, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "/home/jackie/anaconda3/envs/lime-env/lib/python3.7/site-packages/flask/app.py", line 1821, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "/home/jackie/anaconda3/envs/lime-env/lib/python3.7/site-packages/flask/_compat.py", line 39, in reraise
    raise value
  File "/home/jackie/anaconda3/envs/lime-env/lib/python3.7/site-packages/flask/app.py", line 1950, in full_dispatch_request
    rv = self.dispatch_request()
  File "/home/jackie/anaconda3/envs/lime-env/lib/python3.7/site-packages/flask/app